<a href="https://colab.research.google.com/github/BrunoReis136/API_fin_powered_by_OpenAI/blob/main/projeto_fin_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USANDO API DA FINNHUB

## Definir Classe Cliente Finnhub

In [11]:
import requests
import time

# Classe Centralizadora das operações com a API Finnhub
class FinnhubClient:

  # Construtor da Classe recebendo a chave e definindo o url base
  def __init__(self, api_key):
    self.api_key = api_key                      # Chave da API
    self.base_url = 'https://finnhub.io/api/v1' # URL base do Finnhub


  # Definir Método de busca pelo API atravéd de requisição GET
  def _get(self, endpoint, params=None):
    if params is None:
        params = {}

    params["token"] = self.api_key
    url = f"{self.base_url}/{endpoint}"

    try:
        r = requests.get(url, params=params, timeout=10)
        r.raise_for_status()

        # Rate limit do plano free
        time.sleep(0.3)

        return r.json()

    except requests.exceptions.HTTPError as e:
        status = r.status_code

        if status == 403:
            print(f"[403] Acesso negado ao endpoint: {endpoint} (plano ou permissão)")
        elif status == 429:
            print(f"[429] Rate limit excedido no endpoint: {endpoint}")
        else:
            print(f"[HTTP {status}] Erro no endpoint {endpoint}: {r.text}")

        return {"error": status, "endpoint": endpoint}

    except requests.exceptions.Timeout:
        print(f"[TIMEOUT] Endpoint {endpoint} demorou para responder")
        return {"error": "timeout", "endpoint": endpoint}

    except requests.exceptions.RequestException as e:
        print(f"[REQUEST ERROR] {endpoint}: {e}")
        return {"error": "request_exception", "endpoint": endpoint}



  # Define método para retorno das cotação atual do ativo definido (preço, máxima, mínima, abertura, fechamento anterior)
  def quote(self, symbol):
    if not symbol:
      raise ValueError("Symbol não pode ser vazio")
    return self._get(
        "quote",
        {"symbol":symbol} # Código do ativo (AAPL, MSFT, TSLA)
    )

  # Define método pra retorno das candles de um ativo definido em determinado período
  def candles(self, symbol, resolution, start, end):
    if not symbol:
      raise ValueError("Symbol não pode ser vazio")
    return self._get(
        "stock/candle",
        {
            "symbol": symbol,         # Código do ativo (APPL, MSFT, TSLA)
            "resolution": resolution, # Resolução da vela : Em minutos > 1, 5, 15, 30, 60; Período > D (days), W (weeks), M (months)
            "from": start,            # Data inicial
            "to": end                 # Data final
        }
    )


  # Define método para retorno dos rendimentos do ativo definido (real, estimado, surpresa)
  def earnings(self, symbol):
    if not symbol:
      raise ValueError("Symbol não pode ser vazio")
    return self._get(
        "stock/earnings",
        {"symbol":symbol}
    )


  # Define método para retorno dos dados financeiros reportados (demonstração de resultados, balanço patrimonial, fluxo de caixa)
  def financials(self, symbol):
    return self._get(
        "stock/financials-reported",
        {"symbol": symbol}
    )

## Funções de Normalização dos Dados

In [12]:
import pandas as pd

# Função de normalização dos Candles para DataFrame estruturado
def normalize_candles(data):
  try:
    # Confere se a resposta da API indica sucesso
    if not isinstance(data, dict) or data.get("s") != "ok":
      return pd.DataFrame()

    # Verifica se todos os campos necessários existem
    required_keys = {"t", "o", "h", "l", "c", "v"}
    if not required_keys.issubset(data.keys()):
      return pd.DataFrame()

    # Cria um DataFrame a partir de cada array retornado pela API
    df = pd.DataFrame({
        "timestamp": data["t"],  # Tempo em formato UNIX (epoch)
        "open": data["o"],       # Preço de abertura
        "high": data["h"],       # Preço máximo
        "low": data["l"],        # Preço mínimo
        "close": data["c"],      # Preço de fechamento
        "volume": data["v"]      # Volume negociado
    })

    # Converte a coluna data para formato legível
    df["date"] = pd.to_datetime(df["timestamp"], unit="s")

    # Retorna o dataframe excluindo a coluna com tempo em formato bruto
    return df.drop(columns="timestamp")

  except Exception:
    # Qualquer erro inesperado resulta em DataFrame vazio
    return pd.DataFrame()


# Função de normalização direta dos dados de rendimentos
# A API retorna uma lista de dicionários
def normalize_earnings(data):
  try:
    # Verifica se os dados estão no formato esperado (lista)
    if not isinstance(data, list):
      return pd.DataFrame()

    # Converte diretamente para DataFrame
    return pd.DataFrame(data)

  except Exception:
    return pd.DataFrame()


# Função de normalização dos dados financeiros reportados
# Os dados vêm aninhados, então usamos o json_normalize
def normalize_financials(data):
  try:
    # Valida estrutura mínima esperada
    if not isinstance(data, dict) or "data" not in data:
      return pd.DataFrame()

    # Normaliza os dados financeiros
    df = pd.json_normalize(
        data["data"],                 # Lista principal dos relatórios
        record_path=["report", "ic"],  # Caminho até os indicadores financeiros
        meta=[
            ["report", "period"],     # Período do relatório (se existir)
            ["report", "year"],       # Ano fiscal
            ["report", "quarter"]     # Trimestre fiscal
        ],
        errors="ignore"               # Campos ausentes viram NaN
    )

    return df

  except Exception:
    return pd.DataFrame()


## Gerando dados práticos para interpretação através do API da Openai

In [13]:
# Função que adiciona ao DataFrame colunas com valores baseados nos preços
def add_price_features(df):
    required_cols = {"date", "close"}

    if df.empty or not required_cols.issubset(df.columns):
        return df

    df = df.sort_values("date")

    df["return_1d"] = df["close"].pct_change()
    df["ma_7"] = df["close"].rolling(7).mean()
    df["ma_21"] = df["close"].rolling(21).mean()
    df["volatility_21"] = df["return_1d"].rolling(21).std()

    return df


# Função que adiciona ao DataFrame colunas com valores baseados nos 'earnings per share' - 'lucro por ação'
def add_earnings_features(df):
    required_cols = {"actual", "estimate", "surprisePercent"}

    if df.empty or not required_cols.issubset(df.columns):
        return df

    df["eps_diff"] = df["actual"] - df["estimate"]
    df = df.rename(columns={"surprisePercent": "eps_surprise_pct"})

    return df

## Utilização dos recursos criados

In [14]:
# options.py

tickers = [
    # Tecnologia
    "AAPL",   # Apple
    "MSFT",   # Microsoft
    "GOOGL",  # Alphabet (Google)
    "AMZN",   # Amazon
    "META",   # Meta (Facebook)
    "NVDA",   # NVIDIA
    "TSLA",   # Tesla
    "ORCL",   # Oracle
    "ADBE",   # Adobe
    "CRM",    # Salesforce

    # Financeiras
    "JPM",    # JPMorgan Chase
    "BAC",    # Bank of America
    "WFC",    # Wells Fargo
    "GS",     # Goldman Sachs
    "MS",     # Morgan Stanley
    "V",      # Visa
    "MA",     # Mastercard

    # Consumo / Varejo
    "WMT",    # Walmart
    "COST",   # Costco
    "HD",     # Home Depot
    "MCD",    # McDonald's
    "NKE",    # Nike

    # Saúde
    "JNJ",    # Johnson & Johnson
    "PFE",    # Pfizer
    "UNH",    # UnitedHealth
    "ABBV",   # AbbVie
    "MRK",    # Merck

    # Energia
    "XOM",    # Exxon Mobil
    "CVX",    # Chevron

    # Indústria
    "BA",     # Boeing
    "CAT",    # Caterpillar
    "GE"      # General Electric
]

In [15]:
import time
import os
from getpass import getpass


# Busca chave no servidor se em produção ou solicita senha por input
api_key = os.getenv("FINNHUB_API_KEY") or getpass("Insira sua API KEY do Finnhub: ")

# Enunciado da solicitação do Ticker
print("\n\nInsira o ticker/código do ativo a avaliar:\n\nOpções:\n")

for i in range(0, len(tickers), 5):
    print(" , ".join(tickers[i:i+5]))



while True:
  SYMBOL = input('Digite um ticker da lista.\n').strip().upper()
  if SYMBOL in tickers:
    break
  else:
    print('Ticker inválido')



MAX_DAYS = 365 * 5  # 5 anos

while True:
    try:
        period = int(input("Digite o período em dias:\n"))
        if period <= 0 or period > MAX_DAYS:
            raise ValueError
        break
    except ValueError:
        print(f"Digite um número entre 1 e {MAX_DAYS}")

end = int(time.time())
start = end - 60 * 60 * 24 * period

client = FinnhubClient(api_key)

# --- Candles ---
raw_candles = client.candles(SYMBOL, "D", start, end)
df_prices = normalize_candles(raw_candles)
df_prices = add_price_features(df_prices)

# --- Earnings ---
raw_earnings = client.earnings(SYMBOL)
df_earnings = normalize_earnings(raw_earnings)
df_earnings = add_earnings_features(df_earnings)

# --- Financials ---
raw_financials = client.financials(SYMBOL)
df_financials = normalize_financials(raw_financials)

try:
  df_financials.to_excel('financials.xlsx', index=False)
except:
  pass

try:
  df_earnings.to_excel('earnings.xlsx',index=False)
except:
  pass

try:
  df_prices.to_excel('prices.xlsx', index=False)
except:
  pass

print(df_prices.tail())
print(df_earnings.tail())
print(df_financials.head())



Insira sua API KEY do Finnhub: ··········


Insira o ticker/código do ativo a avaliar:

Opções:

AAPL , MSFT , GOOGL , AMZN , META
NVDA , TSLA , ORCL , ADBE , CRM
JPM , BAC , WFC , GS , MS
V , MA , WMT , COST , HD
MCD , NKE , JNJ , PFE , UNH
ABBV , MRK , XOM , CVX , BA
CAT , GE
Digite um ticker da lista.
ABBV
Digite o período em dias:
30
[HTTP 401] Erro no endpoint stock/candle: {"error":"Invalid API key."}
[HTTP 401] Erro no endpoint stock/earnings: {"error":"Invalid API key"}
[HTTP 401] Erro no endpoint stock/financials-reported: {"error":"Invalid API key"}


ValueError: No engine for filetype: ''